# DCGANs

DCGAN means a GAN that uses the convolutional and convolutional-transpose layers in the descriminator and generator models respectively  
1. **Descriminator**: is composed of conv2d, batch normalization layers and uses the LeakyReLU activation with 0.2 as slope. it's input is a 3 x 64 x 64 images and the output is the proba that this image is real (it belongs to the real data distribution) 
2. **Generator** : for the gen, it uses convolutional-transpose and batch normalization layer, with the ReLU activation,the input is a latent vector z and the output is an 3 x 64 x 64  image ( The strided conv-transpose layers allow the latent vector to be transformed into a volume with the same shape as an image)

The Transformed Conv is almost doing the deconvolutional operation , but it doesn't create exctly the inverse results of the convolutional operation

## Import the used packages

In [1]:
from __future__ import print_function
import argparse
import os
import random
import torch as th
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision.utils import make_grid
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib inline
from IPython.display import HTML

We will define some inputs to have with them in all the notebook 

In [2]:


DATA_PATH = '../input/celeba-dataset/img_align_celeba/img_align_celeba'
# number of worker threads for loading the data 
N_WORKERS =  4
# in the DCGAN paper, they used a batch_size of 128 
batch_size = 128
#in this notebook we will use an images of 3 x 64 x 64 (if we desire to change it, we need to changes also in the gen and des architectures)
image_size =64 
nc = 3 # the number of channels (3 for the RGB images)
nz = 100 # length of the latent vector 
ngf = 64 # the depth of the feature maps in  the genegator 
ndf =  64 # the depth of the feature maps in the descriminator 

EPOCHS = 50
lr = 0.0002 # the learning rate for the optimizer (we choose 0.0002 relatively to the paper of the DCGAN)
beta1 = 0.5  # the hyperparameter for the adam optim


We are using the ImageFolder class to extract our data and doing some transformation on it 

we are resizing all the images to the appropriate size that we define earlier and cropping the image on the center, then normalizing it with mean and std

In [3]:
dataset = dset.ImageFolder(root="../input/celeba-dataset/img_align_celeba",
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

KeyboardInterrupt: 

In [ ]:
# creating the dataloader based on the dataset 
dl = th.utils.data.DataLoader(dataset,batch_size = batch_size , shuffle = True,num_workers = N_WORKERS)

In [4]:
# setting the default device 
device = th.device("cuda" if th.cuda.is_available() else 'cpu')

In [5]:
#creating a helper fonction to show some real images samples 
def show_batch(dl): 
    for images, _ in dl : 
        fig, ax = plt.subplots(figsize = (16,10))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images[:20],10).permute(1,2,0))
        break

In [6]:
#plotting some samples 
show_batch(dl)

NameError: name 'dl' is not defined

For the initialization of the model, we are going to use a random weights and bias from the Normale Distribution with mean=0 and std = 0.02 as specified in the DCGAN paper.
for this we are creating the **weight_init** fonction that take a initialized model and re-initialize the weights and the bias of the convolutional, convolutional-transpose and batchNormalize layers with from the normale dist.

In [8]:
def weight_init(model): 
    class_name = model.__class__.__name__
    if class_name.find("Conv") !=-1:  # the convolutional and convolutional-transpose layers
        #nn.init.normal_(tensor) fonction that fils the input tensor with values from the normale distribution 
        nn.init.normal_(model.weight.data,mean=0.0,std=0.02)
    elif  class_name.find("BatchNorm") !=-1: # the batch normalization layer
        nn.init.normal_(model.weight.data, 1.0 , 0.02)
        # nn.init.constant_(tensor , const) fils the input tensor with the constant "const"
        nn.init.constant_(model.bias.data,0)


## Generator

The generator try to map the latent space vector with the data space (real values), and this by creating a RGB images like the training images (3x64x64).

We use 2d convolutional transpose layer with 2d batch normalization and ReLU fonction, and the output is fed through a Tanh fonction to return it into input data ranfge [-1,1]

**for the generator we are going to use the ConvTranspose2d layer which can be seen as the inverse of the conv2d layer (but it's not realy the inverse because it doesn't give as the correct inverse)**

In [ ]:
class Generator(nn.Module): 
    def __init__(self): 
        super(Generator,self).__init__()
        self.main = nn.Sequential(
            # we are going to fit the latent vector into a convolutional
            nn.ConvTranspose2d(nz,ngf*8,4,1,0,bias=False),
            # ngf * 8 is the ouput of the ConvTranspose layer
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),
            # as we have inputs of shape 3 x 64 x 64 so after applying the convTranspose operation we will have
            # output of shape (ngf*8) x 4 x 4  
            
            nn.ConvTranspose2d(ngf * 8 , ngf * 4 , 4 , stride=2,padding=1,bias=False),
            #as we set tha stride =2 and we are appying the ConvTranspose operation 
            # so we will multiply the output shape by 2,contrary to the conv2d operation which devide by the stride
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),
            # output shape will be  (ngf*4) x 8 x 8

            nn.ConvTranspose2d(ngf*4 , ngf*2, 4 , 2, 1,bias=False), # by applying a stride of 2 so the output shape will multiplyed by 2 
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),
            # output shape will be  (ngf*2) x 16 x 16
            
            # we will continue our convTranspose operation until we obtain the shape of our images (3x64x64)
            nn.ConvTranspose2d(ngf*2 , ngf, 4 , 2, 1,bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # output shape will be  ngf x 32 x 32
            
            nn.ConvTranspose2d(ngf,nc, 4 , 2 , 1,bias=False),
            nn.Tanh(),
            # output shape will be  nc x 64 x 64            
        )
    def forward(self,x):
        return self.main(x)
    

Here that we build the Generator network, we will fit it to the weight init fonction so that we initialize the weights from the normal distribution

In [ ]:
Gen =Generator()
print(Gen.main[0].weight.data[0])

In [ ]:
Gen = Generator().to(device)

# Apply the weights_init function to randomly initialize all weights to mean=0, stdev=0.02

Gen.apply(weight_init)

print(Gen)

In [ ]:
Gen.main[0].weight.data[0]

## Descriminator

The descriminator will be a classifier that output the proba that the image is real, for this we are going to use a series of conv2d, batch normalization layers with the Leaky Relu activation fonction with slope of 0.2,and also a sigmoid fonction to convert the outputs to probabilities, we can extend this arcitecture with more layers,

in the DCGAN paper we mentioned that it's a good practise to use strided conv2d layers instead of using the maxpooling and that because it lets the network to learn it's own pooling fonction.

In [9]:
class Descriminator(nn.Module):
    def __init__(self):
        super(Descriminator,self).__init__()
        self.main = nn.Sequential(
        # the input will be an image of : nc x 64 x 64 image
            nn.Conv2d(nc,ndf,4,2,1,bias=False),
            nn.BatchNorm2d(ndf),
            nn.LeakyReLU(0.2), # slop of 0.2 is recommended by the comunity 
            #as we are using the conv2d layers with stride of 2, so the shapes of each channels will be devided by 2 
            # output shape :  ndf x 32 x 32
            
            nn.Conv2d(ndf, ndf*2,4,2,1,bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2),
            #output shape : (ndf*2) x 16 x 16

            nn.Conv2d(ndf*2,ndf*4,4,2,1,bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2),
            # output size : (ndf*4) x 8 x 8
            nn.Conv2d(ndf*4,ndf*8,4,2,1,bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2),
            #output size : ndf*8 x 4 x 4
            nn.Conv2d(ndf*8,1,4,2,1,bias=False),
            nn.Sigmoid(),
        )
    def forward(self,input):
        return self.main(input)


We are going to do the same initialization for the Descriminator 

In [11]:
Des = Descriminator().to(device)

Des.apply(weight_init)

print(Des)


Descriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2)
    (9): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.2)
    (12): Conv2d(512, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1